<a href="https://colab.research.google.com/github/SakthiRishikesh/Cybersec_prjcts/blob/main/streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit pyngrok pandas matplotlib plotly


In [ ]:
%%writefile app.py
"""
Cyber Galaxy Grid — Streamlit Cybersecurity Log Dashboard
Features:
 - Dark, animated celestial background (stars + subtle motion)
 - Embedded/synthetic log generator (no external files required)
 - Search & filters (IP, status, endpoint, time window)
 - IP risk scoring (heuristic + failed-login counting)
 - Top flagged IP cards (neon glass style)
 - Interactive Plotly charts
 - Geo-lookup fallback (local heuristics) + optional API token input
 - Simulated threat feed panel (live-like)
 - CSV export (download current filtered logs)
Requirements: streamlit, pandas, plotly
Run: streamlit run app.py
"""
import streamlit as st
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import plotly.express as px
from io import StringIO

# -------------------------
# Page config + styles
# -------------------------
st.set_page_config(
    page_title="Cyber log analysis",
    layout="wide",
)

# Custom CSS + Animated starfield via HTML component
GALAXY_CSS = """
<style>
/* Background and glass cards */
html, body { height:100%; background: #020617; }
section.main { background: linear-gradient(180deg, rgba(0,6,17,0.8), rgba(1,4,10,0.85)); }

.streamlit-expanderHeader { color: #9ad4ff !important; }
.header-title { font-family: Inter, Roboto, sans-serif; color: #cfeeff; font-weight:800; }
.glass {
    border-radius:12px;
    padding:14px;
    background: linear-gradient(180deg, rgba(255,255,255,0.02), rgba(255,255,255,0.01));
    border: 1px solid rgba(255,255,255,0.04);
    box-shadow: 0 6px 30px rgba(3,6,12,0.6);
    color: #e6f6ff;
}
/* Neon accent badges */
.neon-badge {
    display:inline-block;
    padding:8px 12px;
    border-radius:999px;
    background: linear-gradient(90deg, rgba(0,255,255,0.06), rgba(255,0,255,0.03));
    border: 1px solid rgba(0,255,255,0.08);
    color:#7ef0ff;
    font-weight:700;
}
/* Top card grid */
.top-cards { display:flex; gap:12px; align-items:stretch; }
/* minimal table styling */
[data-testid="stDataFrameContainer"] table { background: transparent !important; color: #dff7ff !important; }
/* small text */
.small-muted { color: #9fb6c9; font-size:13px; }

/* Animated starfield (canvas) */
#starfield {
  position: fixed;
  inset: 0;
  z-index: -1;
  pointer-events: none;
  background: radial-gradient(ellipse at 20% 10%, rgba(97,206,255,0.02), transparent 10%),
              radial-gradient(ellipse at 80% 80%, rgba(209,102,255,0.02), transparent 10%),
              radial-gradient(ellipse at 60% 40%, rgba(255,255,255,0.015), transparent 8%);
}
</style>

<div id="starfield"></div>

<script>
// tiny JS star generator (no external libs) - creates twinkling stars
(function(){
  const n = 80;
  const field = document.getElementById('starfield');
  for(let i=0;i<n;i++){
    const star = document.createElement('div');
    const size = Math.random()*3 + 1;
    star.style.width = size + 'px';
    star.style.height = size + 'px';
    star.style.position = 'absolute';
    star.style.left = (Math.random()*100) + '%';
    star.style.top = (Math.random()*100) + '%';
    star.style.background = 'radial-gradient(circle, #fff 0%, rgba(126,240,255,0.4) 40%, rgba(126,240,255,0.06) 100%)';
    star.style.borderRadius = '50%';
    star.style.filter = 'blur(' + (Math.random()*0.8) + 'px)';
    star.style.opacity = (Math.random()*0.8).toString();
    star.style.transform = 'translate3d(0,0,0)';
    field.appendChild(star);
  }
  // subtle drifting animation
  let t = 0;
  function anim(){ t += 0.002; field.style.transform = 'translateY(' + Math.sin(t)*8 + 'px)'; requestAnimationFrame(anim); }
  anim();
})();
</script>
"""
st.markdown(GALAXY_CSS, unsafe_allow_html=True)

st.markdown("<div style='display:flex;align-items:center;gap:12px'><div style='font-size:28px'></div><div class='header-title' style='font-size:22px'>Cyber Log Analysis — Live SOC Dashboard</div></div>", unsafe_allow_html=True)
st.markdown("<div class='small-muted'>Dark cyber-galaxy theme • Neon glass UI • Risk scoring • Geo lookup (optional API)</div>", unsafe_allow_html=True)
st.markdown("---")

# -------------------------
# Synthetic log generation (no external files)
# -------------------------
@st.cache_data
def generate_synthetic_logs(n=5000, hours=6):
    start = datetime.utcnow() - timedelta(hours=hours)
    # generate set of IPs (some public, some private)
    public_ips = [f"198.51.100.{i}" for i in range(2, 20)] + [f"203.0.113.{i}" for i in range(2, 12)]
    private_ips = [f"192.168.1.{i}" for i in range(2, 12)] + [f"10.0.0.{i}" for i in range(2, 8)]
    ips = public_ips + private_ips
    statuses = [200]*12 + [404]*2 + [401]*3 + [500]*1
    methods = ["GET", "POST", "PUT"]
    endpoints = ["/", "/login", "/api/data", "/admin", "/wp-login.php"]
    rows = []
    for _ in range(n):
        ip = random.choice(ips)
        ts = start + timedelta(seconds=random.randint(0, hours*3600))
        method = random.choice(methods)
        endpoint = random.choice(endpoints)
        status = random.choice(statuses)
        size = random.randint(120, 8000)
        rows.append((ip, ts, method, endpoint, status, size))
    df = pd.DataFrame(rows, columns=["ip", "timestamp", "method", "endpoint", "status", "size"])
    df['timestamp'] = pd.to_datetime(df['timestamp']).dt.tz_localize(None)
    return df.sort_values('timestamp').reset_index(drop=True)

df = generate_synthetic_logs(n=6000, hours=8)

# -------------------------
# Sidebar controls: filters, geo token, options
# -------------------------
with st.sidebar:
    st.markdown("<div class='glass' style='padding:12px'>", unsafe_allow_html=True)
    st.markdown("### 🔎 Filters & Controls", unsafe_allow_html=True)
    ip_filter = st.text_input("IP contains (search)", "")
    status_filter = st.multiselect("Status codes", options=sorted(df['status'].unique()), default=[])
    endpoint_filter = st.text_input("Endpoint contains", "")
    time_window = st.slider("Show last N hours", 1, 24, 8)
    st.markdown("#### 🌐 Geo lookup (optional)")
    geo_api_key = st.text_input("Geo API token (optional)", "", help="If you have an external geo API (ipinfo, ipstack, etc.) paste token to enable real lookups.")
    st.markdown("---")
    st.markdown("### ⚙️ Detection tuning")
    failed_word_weight = st.slider("Failed-word weight", 1, 5, 3)
    status_weights = {
        '200': st.slider("200 weight", 0, 5, 0),
        '401': st.slider("401 weight", 0, 10, 4),
        '404': st.slider("404 weight", 0, 10, 3),
        '500': st.slider("500 weight", 0, 10, 6),
    }
    st.markdown("</div>", unsafe_allow_html=True)

# Apply filters
cutoff_time = datetime.utcnow() - timedelta(hours=time_window)
df_filtered = df[df['timestamp'] >= cutoff_time]

if ip_filter:
    df_filtered = df_filtered[df_filtered['ip'].str.contains(ip_filter, na=False)]
if status_filter:
    df_filtered = df_filtered[df_filtered['status'].isin(status_filter)]
if endpoint_filter:
    df_filtered = df_filtered[df_filtered['endpoint'].str.contains(endpoint_filter, na=False)]

# -------------------------
# IP Risk Scoring heuristic
# -------------------------
def compute_risk_scores(df_in):
    # base score per event from status
    status_score_map = {200: status_weights.get('200', 0), 401: status_weights.get('401', 4),
                        404: status_weights.get('404', 3), 500: status_weights.get('500', 6)}
    dfc = df_in.copy()
    dfc['status_weight'] = dfc['status'].map(status_score_map).fillna(1)
    dfc['failed_word'] = dfc['endpoint'].str.contains('login|admin|wp-login', case=False, na=False).astype(int)
    dfc['failed_word'] = dfc['failed_word'] * failed_word_weight
    # aggregate by IP
    agg = dfc.groupby('ip').agg(
        events=('ip', 'size'),
        total_score=('status_weight', 'sum'),
        failed_word_count=('failed_word', 'sum'),
        last_seen=('timestamp', 'max'),
        event_count=('status', 'count')
    ).reset_index()
    # normalize and final score
    agg['score'] = agg['total_score'] + agg['failed_word_count'] * 1.25
    # suspicious flag threshold (dynamic)
    thresh = max(5, np.percentile(agg['score'].values, 85) if len(agg) > 0 else 5)
    agg['suspicious'] = agg['score'] >= thresh
    agg = agg.sort_values('score', ascending=False)
    return agg

ip_stats = compute_risk_scores(df_filtered)

# -------------------------
# Top summary row (cards)
# -------------------------
col_a, col_b, col_c, col_d = st.columns([2,2,2,3])
col_a.markdown(f"<div class='glass'><h4>Total Events</h4><h2>{len(df_filtered):,}</h2></div>", unsafe_allow_html=True)
col_b.markdown(f"<div class='glass'><h4>Unique IPs</h4><h2>{df_filtered['ip'].nunique():,}</h2></div>", unsafe_allow_html=True)
col_c.markdown(f"<div class='glass'><h4>Suspicious IPs</h4><h2>{ip_stats['suspicious'].sum()}</h2></div>", unsafe_allow_html=True)

## ----------------------------------
# 🚨 Suspicious IP Insight Cards (New Clean UI)
# ----------------------------------

st.markdown("## 🚨 Suspicious Activity Overview")

sus = ip_stats[ip_stats['suspicious']]

if sus.empty:
    st.success("No suspicious IPs detected ✅")
else:
    sus_display = sus[['ip', 'score', 'event_count', 'last_seen']].head(6)

    for _, row in sus_display.iterrows():
        with st.container():
            st.markdown(
                f"""
                <div style="
                    padding:14px;
                    margin:8px 0;
                    border-radius:12px;
                    background:rgba(0,0,0,0.35);
                    border:1px solid rgba(255,255,255,0.15);
                    box-shadow:0 0 12px rgba(0,255,255,0.4);
                ">
                    <h4 style="color:#00eaff;margin:0;">{row['ip']}</h4>
                    <p style="margin:4px 0 0 0;color:#cfeeff;">
                        Risk Score: <b style="color:#ffdf6a;">{row['score']:.1f}</b><br>
                        Event Count: <b style="color:#ff7fa9;">{row['event_count']}</b><br>
                        Last Seen: <span style="color:#d0d0d0;">{pd.to_datetime(row['last_seen']).strftime('%Y-%m-%d %H:%M:%S')}</span>
                    </p>
                </div>
                """,
                unsafe_allow_html=True
            )


# -------------------------
# Charts: top IPs and timeline
# -------------------------
left_col, right_col = st.columns([3,2])
with left_col:
    top_ips = df_filtered['ip'].value_counts().reset_index()
    top_ips.columns = ['ip', 'count']
    fig = px.bar(top_ips.head(12), x='ip', y='count', title='Top IPs by Volume', text='count')
    fig.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
    st.plotly_chart(fig, use_container_width=True)

    # timeline (1H bins)
    timeline = df_filtered.set_index('timestamp').resample('1H').size().reset_index(name='events')
    fig2 = px.line(timeline, x='timestamp', y='events', title='Events Over Time (1H bins)', markers=True)
    fig2.update_layout(xaxis_title='', yaxis_title='Events', plot_bgcolor='rgba(0,0,0,0)')
    st.plotly_chart(fig2, use_container_width=True)

with right_col:
    st.markdown("<div class='glass'><h4>Suspicious IP Breakdown</h4>", unsafe_allow_html=True)
    sus = ip_stats[ip_stats['suspicious']]
    if not sus.empty:
        st.table(sus[['ip', 'score', 'event_count']].head(10).style.format({'score':'{:.1f}'}))
    else:
        st.markdown("<div class='small-muted'>No flagged IPs — good job!</div>", unsafe_allow_html=True)
    st.markdown("</div>", unsafe_allow_html=True)

# -------------------------
# IP Drilldown interactive
# -------------------------
st.markdown("### 🔬 IP Drilldown")
ips_for_select = ip_stats['ip'].tolist()
sel_ip = st.selectbox("Select IP", options=ips_for_select, index=0 if ips_for_select else None)
if sel_ip:
    detail = df_filtered[df_filtered['ip'] == sel_ip].sort_values('timestamp', ascending=False)
    st.markdown(f"<div class='glass'><b>Selected IP:</b> {sel_ip} • <span class='small-muted'>Events: {len(detail)}</span></div>", unsafe_allow_html=True)
    # small table and trend chart
    tcnt = detail.set_index('timestamp').resample('30min').size().reset_index(name='events')
    fig3 = px.area(tcnt, x='timestamp', y='events', title=f'Activity Trend for {sel_ip}')
    st.plotly_chart(fig3, use_container_width=True)
    st.dataframe(detail.head(50))

# -------------------------
# Geo lookup stub + optional real API
# -------------------------
st.markdown("### 🌍 Geo Lookup (Optional)")
def simple_geo(ip):
    # Very small local heuristic mapping for demonstration
    if ip.startswith("192.168") or ip.startswith("10."):
        return {"country": "Private", "city": "-", "asn": "LAN"}
    if ip.startswith("198.51.100"):
        return {"country": "US", "city": "Newark", "asn": "AS65500"}
    if ip.startswith("203.0.113"):
        return {"country": "AU", "city": "Sydney", "asn": "AS65501"}
    # fallback
    return {"country": "Unknown", "city": "-", "asn": "AS-UNK"}

if st.button("Lookup selected IP geo (local heuristic)"):
    g = simple_geo(sel_ip)
    st.markdown(f"<div class='glass'><b>{sel_ip}</b> • Country: {g['country']} • City: {g['city']} • ASN: {g['asn']}</div>", unsafe_allow_html=True)

if geo_api_key:
    st.markdown("<div class='small-muted'>Real API key provided — external lookup would be attempted here if integrated.</div>", unsafe_allow_html=True)

# -------------------------
# Threat feed (simulated)
# -------------------------
st.markdown("### 🛰️ Threat Feed — Recent Alerts")
feed_col1, feed_col2 = st.columns(2)
with feed_col1:
    st.markdown("<div class='glass'><b>Feed (Live-ish)</b></div>", unsafe_allow_html=True)
    # Simulate some feed items
    sample_alerts = [
        {"time": "Just now", "title": "Brute force attempts detected", "detail": "Multiple 401s from 203.0.113.9"},
        {"time": "5m ago", "title": "SQLi pattern", "detail": "Suspicious payload detected at /api/data"},
        {"time": "30m ago", "title": "Mass 404s", "detail": "High 404 rate from 198.51.100.14"},
    ]
    for a in sample_alerts:
        st.markdown(f"<div class='glass' style='margin-bottom:8px'><b>{a['title']}</b> <div class='small-muted'>{a['time']} • {a['detail']}</div></div>", unsafe_allow_html=True)

with feed_col2:
    st.markdown("<div class='glass'><b>Mitigation Tips</b></div>", unsafe_allow_html=True)
    st.markdown("<ul class='small-muted'><li>Block IPs with sustained high risk score</li><li>Enable rate-limiting on /login</li><li>Enrich logs with ASN and GEO for prioritization</li></ul>", unsafe_allow_html=True)

# -------------------------
# Logs table + CSV download
# -------------------------
st.markdown("### 📄 Current Filtered Logs")
st.download_button("📥 Download filtered logs (CSV)", data=df_filtered.to_csv(index=False), file_name="filtered_logs.csv", mime="text/csv")
st.dataframe(df_filtered.head(200))

# -------------------------
# Footer
# -------------------------
st.markdown("---")
st.markdown("<div class='small-muted'>Built with 🌌 Cyber Galaxy Grid • Demo heuristics only — replace geo/API integration for production.</div>", unsafe_allow_html=True)


Overwriting app.py


In [ ]:
!pip install streamlit pyngrok


In [ ]:
!ngrok config add-authtoken  35Go4gm6M6BUAg62sJD1Tu01TCc_2SRGwNNoNkDKp7vxYTVX2

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install pyngrok

from pyngrok import ngrok

# Set your token
ngrok.set_auth_token("35Go4gm6M6BUAg62sJD1Tu01TCc_2SRGwNNoNkDKp7vxYTVX2")

# Run Streamlit in background
!streamlit run app.py --server.port 8501 &

# Create public URL
public_url = ngrok.connect(8501)
print("Your Streamlit app is running at:", public_url)



2025-11-10 03:34:30.747 Port 8501 is already in use
Your Streamlit app is running at: NgrokTunnel: "https://claribel-untwilled-nakisha.ngrok-free.dev" -> "http://localhost:8501"
